In [5]:
import numpy as np
import math
import itertools
import heapq
from dataclasses import dataclass
from typing import List, Dict, Tuple

# ----------------------------
# 1) 데이터 구조
# ----------------------------
@dataclass(frozen=True)
class Employee:
    name: str
    trust: float
    communication: float
    decision_making: float
    innovative_thinking: float
    conflict_management: float
    role_clarity: float
    psychological_safety: float

_METRICS = [
    "trust", "communication", "decision_making",
    "innovative_thinking", "conflict_management",
    "role_clarity", "psychological_safety"
]

_DRIVER_KO = {
    "trust": "신뢰",
    "communication": "커뮤니케이션",
    "innovative_thinking": "혁신/아이디어",
    "decision_making": "의사결정 균형",
    "conflict_management": "갈등관리",
    "role_clarity": "역할명확성",
    "psychological_safety": "심리적 안전",
}


 #샘플 데이터: 50명 (E001~E050)
# ----------------------------
SAMPLE_EMPLOYEES_50: List[Employee] = [
    Employee(name="E001", trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99),
    Employee(name="E002", trust=0.99, communication=0.99, decision_making=0.48, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99),
    Employee(name="E003", trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99),
    Employee(name="E004", trust=0.99, communication=0.99, decision_making=0.44, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychological_safety=0.99),
    Employee(name="E005", trust=0.88, communication=0.80, decision_making=0.58, innovative_thinking=0.85, conflict_management=0.82, role_clarity=0.74, psychological_safety=0.79),
    Employee(name="E006", trust=0.76, communication=0.70, decision_making=0.46, innovative_thinking=0.47, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.66),
    Employee(name="E007", trust=0.84, communication=0.79, decision_making=0.50, innovative_thinking=0.56, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),
    Employee(name="E008", trust=0.81, communication=0.76, decision_making=0.53, innovative_thinking=0.52, conflict_management=0.75, role_clarity=0.87, psychological_safety=0.71),
    Employee(name="E009", trust=0.79, communication=0.73, decision_making=0.49, innovative_thinking=0.51, conflict_management=0.72, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E010", trust=0.86, communication=0.82, decision_making=0.56, innovative_thinking=0.58, conflict_management=0.80, role_clarity=0.91, psychological_safety=0.77),
    Employee(name="E011", trust=0.83, communication=0.77, decision_making=0.47, innovative_thinking=0.53, conflict_management=0.74, role_clarity=0.88, psychological_safety=0.72),
    Employee(name="E012", trust=0.77, communication=0.71, decision_making=0.45, innovative_thinking=0.48, conflict_management=0.70, role_clarity=0.84, psychological_safety=0.67),
    Employee(name="E013", trust=0.87, communication=0.84, decision_making=0.57, innovative_thinking=0.59, conflict_management=0.81, role_clarity=0.92, psychological_safety=0.78),
    Employee(name="E014", trust=0.82, communication=0.75, decision_making=0.51, innovative_thinking=0.55, conflict_management=0.76, role_clarity=0.87, psychological_safety=0.70),
    Employee(name="E015", trust=0.80, communication=0.74, decision_making=0.43, innovative_thinking=0.46, conflict_management=0.73, role_clarity=0.86, psychological_safety=0.68),
    Employee(name="E016", trust=0.85, communication=0.80, decision_making=0.54, innovative_thinking=0.58, conflict_management=0.78, role_clarity=0.90, psychological_safety=0.75),
    Employee(name="E017", trust=0.78, communication=0.72, decision_making=0.50, innovative_thinking=0.49, conflict_management=0.71, role_clarity=0.85, psychological_safety=0.69),
    Employee(name="E018", trust=0.88, communication=0.83, decision_making=0.55, innovative_thinking=0.60, conflict_management=0.82, role_clarity=0.93, psychological_safety=0.80),
    Employee(name="E019", trust=0.76, communication=0.70, decision_making=0.47, innovative_thinking=0.45, conflict_management=0.69, role_clarity=0.83, psychological_safety=0.65),
    Employee(name="E020", trust=0.84, communication=0.79, decision_making=0.53, innovative_thinking=0.57, conflict_management=0.77, role_clarity=0.89, psychological_safety=0.74),

    Employee(name="E021", trust=0.70, communication=0.62, decision_making=0.30, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.52, psychological_safety=0.86),
    Employee(name="E022", trust=0.66, communication=0.58, decision_making=0.78, innovative_thinking=0.90, conflict_management=0.53, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E023", trust=0.72, communication=0.70, decision_making=0.45, innovative_thinking=0.84, conflict_management=0.60, role_clarity=0.58, psychological_safety=0.80),
    Employee(name="E024", trust=0.60, communication=0.55, decision_making=0.82, innovative_thinking=0.92, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E025", trust=0.75, communication=0.68, decision_making=0.38, innovative_thinking=0.86, conflict_management=0.62, role_clarity=0.60, psychological_safety=0.83),
    Employee(name="E026", trust=0.63, communication=0.57, decision_making=0.70, innovative_thinking=0.89, conflict_management=0.54, role_clarity=0.50, psychological_safety=0.88),
    Employee(name="E027", trust=0.78, communication=0.73, decision_making=0.55, innovative_thinking=0.81, conflict_management=0.68, role_clarity=0.63, psychological_safety=0.78),
    Employee(name="E028", trust=0.65, communication=0.60, decision_making=0.25, innovative_thinking=0.90, conflict_management=0.52, role_clarity=0.47, psychological_safety=0.92),
    Employee(name="E029", trust=0.69, communication=0.64, decision_making=0.66, innovative_thinking=0.87, conflict_management=0.56, role_clarity=0.54, psychological_safety=0.85),
    Employee(name="E030", trust=0.74, communication=0.71, decision_making=0.48, innovative_thinking=0.83, conflict_management=0.61, role_clarity=0.59, psychological_safety=0.81),
    Employee(name="E031", trust=0.62, communication=0.56, decision_making=0.80, innovative_thinking=0.91, conflict_management=0.49, role_clarity=0.45, psychological_safety=0.90),
    Employee(name="E032", trust=0.77, communication=0.72, decision_making=0.35, innovative_thinking=0.85, conflict_management=0.67, role_clarity=0.62, psychological_safety=0.82),
    Employee(name="E033", trust=0.68, communication=0.63, decision_making=0.60, innovative_thinking=0.88, conflict_management=0.55, role_clarity=0.53, psychological_safety=0.87),
    Employee(name="E034", trust=0.61, communication=0.55, decision_making=0.74, innovative_thinking=0.92, conflict_management=0.51, role_clarity=0.48, psychological_safety=0.89),
    Employee(name="E035", trust=0.76, communication=0.74, decision_making=0.42, innovative_thinking=0.84, conflict_management=0.66, role_clarity=0.64, psychological_safety=0.79),
    Employee(name="E036", trust=0.64, communication=0.59, decision_making=0.83, innovative_thinking=0.90, conflict_management=0.50, role_clarity=0.46, psychological_safety=0.91),
    Employee(name="E037", trust=0.71, communication=0.66, decision_making=0.58, innovative_thinking=0.86, conflict_management=0.57, role_clarity=0.55, psychological_safety=0.84),
    Employee(name="E038", trust=0.67, communication=0.61, decision_making=0.28, innovative_thinking=0.89, conflict_management=0.52, role_clarity=0.49, psychological_safety=0.92),
    Employee(name="E039", trust=0.73, communication=0.69, decision_making=0.72, innovative_thinking=0.82, conflict_management=0.60, role_clarity=0.57, psychological_safety=0.80),
    Employee(name="E040", trust=0.65, communication=0.60, decision_making=0.50, innovative_thinking=0.87, conflict_management=0.54, role_clarity=0.51, psychological_safety=0.88),

    Employee(name="E041", trust=0.62, communication=0.70, decision_making=0.88, innovative_thinking=0.68, conflict_management=0.55, role_clarity=0.50, psychological_safety=0.60),
    Employee(name="E042", trust=0.58, communication=0.66, decision_making=0.92, innovative_thinking=0.72, conflict_management=0.50, role_clarity=0.48, psychological_safety=0.58),
    Employee(name="E043", trust=0.66, communication=0.73, decision_making=0.80, innovative_thinking=0.65, conflict_management=0.60, role_clarity=0.52, psychological_safety=0.62),
    Employee(name="E044", trust=0.55, communication=0.61, decision_making=0.95, innovative_thinking=0.75, conflict_management=0.48, role_clarity=0.45, psychological_safety=0.55),
    Employee(name="E045", trust=0.70, communication=0.76, decision_making=0.78, innovative_thinking=0.63, conflict_management=0.62, role_clarity=0.55, psychological_safety=0.65),
    Employee(name="E046", trust=0.60, communication=0.68, decision_making=0.85, innovative_thinking=0.70, conflict_management=0.53, role_clarity=0.49, psychological_safety=0.59),
    Employee(name="E047", trust=0.64, communication=0.72, decision_making=0.90, innovative_thinking=0.74, conflict_management=0.56, role_clarity=0.51, psychological_safety=0.61),
    Employee(name="E048", trust=0.57, communication=0.63, decision_making=0.82, innovative_thinking=0.67, conflict_management=0.49, role_clarity=0.46, psychological_safety=0.56),
    Employee(name="E049", trust=0.71, communication=0.78, decision_making=0.76, innovative_thinking=0.62, conflict_management=0.63, role_clarity=0.56, psychological_safety=0.67),
    Employee(name="E050", trust=0.59, communication=0.67, decision_making=0.93, innovative_thinking=0.73, conflict_management=0.52, role_clarity=0.48, psychological_safety=0.58),
]

# ----------------------------
# 2) 기본 함수
# ----------------------------
def mean(vals) -> float:
    return float(np.mean(vals))

def diversity(vals) -> float:
    return float(np.std(vals))

def balance_score(std: float, target: float, tolerance: float) -> float:
    return float(np.exp(-((std - target) ** 2) / (2 * (tolerance ** 2))))

# ----------------------------
# 3) weights 검증 (외부 주입용 안전장치)
# ----------------------------
def validate_weights(weights: Dict[str, float], require_sum_to_one: bool = True) -> Dict[str, float]:
    missing = [k for k in _METRICS if k not in weights]
    extra = [k for k in weights.keys() if k not in _METRICS]
    if missing:
        raise ValueError(f"weights에 누락된 키가 있음: {missing}")
    if extra:
        raise ValueError(f"weights에 불필요한 키가 있음: {extra}")

    w = {k: float(weights[k]) for k in _METRICS}
    if any(v < 0 for v in w.values()):
        raise ValueError("weights는 음수가 될 수 없음")

    s = sum(w.values())
    if require_sum_to_one:
        # 외부 산출치가 1로 딱 맞지 않을 수 있으니, 보정 or 에러 중 택1
        if not np.isclose(s, 1.0, atol=1e-6):
            raise ValueError(f"weights 합이 1이 아님: sum={s:.6f}")
    return w
# ----------------------------
# 4) 팀 성향(가중치 미적용) + 팀 점수(가중치 적용) 분리
# ----------------------------
def team_profile_raw(
    team: List[Employee],
    dec_target: float = 0.12,
    dec_tolerance: float = 0.10,
) -> Tuple[Dict[str, float], Dict[str, str], float]:
    """
    ✅ 가중치 미적용 '팀 성향' 산출:
      - 6개: 평균(mean)
      - decision_making: 균형점수(balance_score) 0~1
    returns: (drivers_raw, notes, dec_std)
    """
    trust = [m.trust for m in team]
    comm  = [m.communication for m in team]
    dec   = [m.decision_making for m in team]
    innov = [m.innovative_thinking for m in team]
    conf  = [m.conflict_management for m in team]
    role  = [m.role_clarity for m in team]
    psy   = [m.psychological_safety for m in team]

    trust_mean = mean(trust)
    comm_mean  = mean(comm)
    innov_mean = mean(innov)
    conf_mean  = mean(conf)
    psy_mean   = mean(psy)
    role_mean  = mean(role)

    dec_std = diversity(dec)
    dec_balance = balance_score(dec_std, target=dec_target, tolerance=dec_tolerance)

    drivers_raw = {
        "trust": trust_mean,
        "communication": comm_mean,
        "innovative_thinking": innov_mean,
        "decision_making": dec_balance,  # 0~1
        "conflict_management": conf_mean,
        "role_clarity": role_mean,
        "psychological_safety": psy_mean
    }

    notes: Dict[str, str] = {}

    def explain_mean(name, val):
        if val >= 0.75: return f"{name} 평균이 높음(≈{val:.2f})."
        if val >= 0.55: return f"{name} 평균이 보통(≈{val:.2f})."
        return f"{name} 평균이 낮음(≈{val:.2f})."

    notes["trust"] = explain_mean("Trust", trust_mean)
    notes["communication"] = explain_mean("Communication", comm_mean)
    notes["innovative_thinking"] = explain_mean("Innovative thinking", innov_mean)
    notes["conflict_management"] = explain_mean("Conflict management", conf_mean)
    notes["role_clarity"] = explain_mean("Role clarity", role_mean)
    notes["psychological_safety"] = explain_mean("Psychological safety", psy_mean)

    # decision_making 해석(표준편차 기반)
    if dec_balance >= 0.75:
        notes["decision_making"] = f"의사결정 다양성 적정(표준편차≈{dec_std:.2f})."
    elif dec_std < 0.06:
        notes["decision_making"] = f"의사결정이 너무 유사(표준편차≈{dec_std:.2f}) → 경직 위험."
    else:
        notes["decision_making"] = f"의사결정 다양성 큼(표준편차≈{dec_std:.2f}) → 조율 비용 증가 가능."

    return drivers_raw, notes, dec_std


def team_score_weighted(
    drivers_raw: Dict[str, float],
    weights: Dict[str, float],
) -> float:
    """
    ✅ 가중치 적용 '업무 적합 점수' 산출 (0~100)
    """
    w = validate_weights(weights, require_sum_to_one=True)
    total = sum(w[k] * drivers_raw[k] for k in _METRICS)
    return float(np.clip(total * 100.0, 0, 100))


# ----------------------------
# 5) 강점/리스크 요약: ✅ 가중치 미적용(drivers_raw)로만 판단
# ----------------------------
def pick_strengths_and_risks_raw(
    drivers_raw: Dict[str, float],
    strengths_n: int = 2,
    risks_n: int = 1,
    exclude_from_strengths: Tuple[str, ...] = ("decision_making",),  # 필요 시 빈 튜플로
    exclude_from_risks: Tuple[str, ...] = (),  # 필요 시 ("decision_making",) 등
) -> Tuple[List[str], List[str]]:
    """
    강점/리스크를 '순수 성향 점수'로만 선정
    - 강점: drivers_raw 높은 순
    - 리스크: drivers_raw 낮은 순
    """
    items = [(k, drivers_raw[k]) for k in _METRICS]

    strength_pool = [(k, v) for k, v in items if k not in exclude_from_strengths]
    risk_pool     = [(k, v) for k, v in items if k not in exclude_from_risks]

    strengths = [k for k, _ in sorted(strength_pool, key=lambda x: x[1], reverse=True)[:strengths_n]]
    risks     = [k for k, _ in sorted(risk_pool, key=lambda x: x[1])[:risks_n]]

    return strengths, risks


def _team_brief_raw(
    drivers_raw: Dict[str, float],
    notes: Dict[str, str],
    strengths_n: int = 2,
    risks_n: int = 1,
) -> str:
    strengths, risks = pick_strengths_and_risks_raw(
        drivers_raw,
        strengths_n=strengths_n,
        risks_n=risks_n,
        exclude_from_strengths=("decision_making",),  # 기존처럼 의사결정은 '해석 문장'으로만
        exclude_from_risks=(),  # 리스크에는 포함 가능(원하면 ("decision_making",)로)
    )

    s_txt = ", ".join([f"{_DRIVER_KO[k]}(≈{drivers_raw[k]:.2f})" for k in strengths])
    r_txt = ", ".join([f"{_DRIVER_KO[k]}(≈{drivers_raw[k]:.2f})" for k in risks])

    return f"강점: {s_txt} / 리스크: {r_txt}. {notes['decision_making']}"


# ----------------------------
# 6) Top-K 팀 전수 평가: 점수는 가중치 적용 / 요약은 가중치 미적용
# ----------------------------
def topk_teams_exact(
    employees: List[Employee],
    team_size: int,
    weights: Dict[str, float],
    top_k: int = 5,
    log_every: int = 50_000,
) -> List[Dict]:

    w = validate_weights(weights, require_sum_to_one=True)

    n = len(employees)
    if team_size < 2 or team_size > n:
        raise ValueError(f"team_size={team_size} must be between 2 and len(employees)={n}")

    comb_count = math.comb(n, team_size)
    effective_top_k = min(top_k, comb_count)

    print(f"[INFO] employees={n}, team_size={team_size}, combinations={comb_count:,} (exact)")

    heap: List[Tuple[float, Tuple[int, ...]]] = []

    for c, idx in enumerate(itertools.combinations(range(n), team_size), start=1):
        team = [employees[i] for i in idx]
        drivers_raw, notes, _ = team_profile_raw(team)
        score = team_score_weighted(drivers_raw, w)

        if len(heap) < effective_top_k:
            heapq.heappush(heap, (score, idx))
        elif score > heap[0][0]:
            heapq.heapreplace(heap, (score, idx))

        if log_every and (c % log_every == 0):
            best_now = max(heap, key=lambda x: x[0])[0] if heap else 0.0
            print(f"[PROGRESS] {c:,}/{comb_count:,} checked | best≈{best_now:.2f}")

    winners = sorted(heap, key=lambda x: x[0], reverse=True)

    results: List[Dict] = []
    for rank, (score, idx) in enumerate(winners, start=1):
        team = [employees[i] for i in idx]
        drivers_raw, notes, _ = team_profile_raw(team)
        score2 = team_score_weighted(drivers_raw, w)

        results.append({
            "rank": rank,
            "score": round(score2, 2),  # ✅ 가중치 적용 점수(랭킹용)
            "members": [m.name for m in team],
            "brief": _team_brief_raw(drivers_raw, notes, strengths_n=2, risks_n=1),  # ✅ 가중치 미적용 요약
            "driver_scores_team_raw": {k: round(v, 3) for k, v in drivers_raw.items()},  # ✅ 순수 성향
        })
    return results


def print_top_teams_with_individuals(
    top_teams: List[Dict],
    employees: List[Employee],
    weights: Dict[str, float],
) -> None:

    w = validate_weights(weights, require_sum_to_one=True)

    print(f"\n=== TOP {len(top_teams)} Teams ===")
    print(f"[WEIGHTS] " + ", ".join([f"{k}={w[k]:.3f}" for k in _METRICS]))

    name_to_emp = {e.name: e for e in employees}

    for t in top_teams:
        print(f"\n#{t['rank']} | TeamScore={t['score']:.2f} | Members={', '.join(t['members'])}")
        print(f"- 요약(가중치 미적용): {t['brief']}")
        print(f"- Team driver scores (raw): {t['driver_scores_team_raw']}")

        print("  [Individuals | raw & weighted]")
        for member_name in t["members"]:
            emp = name_to_emp[member_name]
            raw, weighted, total_100 = individual_weighted_breakdown(emp, w)

            raw_str = ", ".join([f"{k}={raw[k]:.2f}" for k in _METRICS])
            wtd_str = ", ".join([f"{k}={weighted[k]:.3f}" for k in _METRICS])

            print(f"  - {emp.name} | indiv_total≈{total_100:.2f}")
            print(f"    raw: {raw_str}")
            print(f"    wtd: {wtd_str}")


EXTERNAL_WEIGHTS = { "innovative_thinking": 0.26, "decision_making": 0.20, "communication": 0.18, "trust": 0.16,
            "psychological_safety": 0.08, "conflict_management": 0.07, "role_clarity": 0.05 }  # 합=1
top5 = topk_teams_exact(employees=SAMPLE_EMPLOYEES_50, team_size=4, weights=EXTERNAL_WEIGHTS, top_k=5)
print_top_teams_with_individuals(top5, SAMPLE_EMPLOYEES_50, EXTERNAL_WEIGHTS)


[INFO] employees=50, team_size=4, combinations=230,300 (exact)
[PROGRESS] 50,000/230,300 checked | best≈94.54
[PROGRESS] 100,000/230,300 checked | best≈94.54
[PROGRESS] 150,000/230,300 checked | best≈94.54
[PROGRESS] 200,000/230,300 checked | best≈94.54

=== TOP 5 Teams ===
[WEIGHTS] trust=0.160, communication=0.180, decision_making=0.200, innovative_thinking=0.260, conflict_management=0.070, role_clarity=0.050, psychological_safety=0.080

#1 | TeamScore=94.54 | Members=E001, E002, E003, E004
- 요약(가중치 미적용): 강점: 신뢰(≈0.99), 커뮤니케이션(≈0.99) / 리스크: 의사결정 균형(≈0.77). 의사결정 다양성 적정(표준편차≈0.05).
- Team driver scores (raw): {'trust': 0.99, 'communication': 0.99, 'innovative_thinking': 0.99, 'decision_making': 0.767, 'conflict_management': 0.99, 'role_clarity': 0.99, 'psychological_safety': 0.99}
  [Individuals | raw & weighted]
  - E001 | indiv_total≈90.20
    raw: trust=0.99, communication=0.99, decision_making=0.55, innovative_thinking=0.99, conflict_management=0.99, role_clarity=0.99, psychologica